In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.plot_functions import *

In [ ]:
resultados = {
              "Baseline Sersic OIDS Resnet18": load__preds("delight_baseline_resnet18"),
              "Baseline Sersic OIDS Resnet18 dataset[x30] BS512": load__preds("delight_augmented_x30_resnet18"),
              "Baseline Sersic OIDS": load__preds("delight_baseline_sersic_oids"),
              "Augmented dataset[x30] BS 512": load__preds("delight_augmented_x30_sersic_oids"),
              "Augmented dataset[x30] balanced v2 log": load__preds("delight_augmented_x30_sersic_oids_balanced_log_01_50"),

              }

In [ ]:
plot_residuals_vs_host_dist(resultados, experiments=["Baseline Sersic OIDS",
                                                     "Augmented dataset[x30] BS 512",
                                                     "Augmented dataset[x30] balanced v2 log"],
                                            
                                        titles = ["Baseline","Autolabeling", "Autolabeling log-balanced"],
                                        cmap_name ="cool",
                                        dpi=300)

In [ ]:
for title, _ in resultados.items():
    print(title.upper())
    print_metrics(resultados, title)
    print("")